<a href="https://colab.research.google.com/github/amrzhd/EEGNet/blob/main/EEGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motor Imagery Task Classification

# Extracting Data

##Downloading BCI Competition IV 2a Dataset

In [ ]:
!wget https://www.bbci.de/competition/download/competition_iv/BCICIV_2a_gdf.zip

In [ ]:
!mkdir -p /content/cleaned_data/

In [ ]:
%%capture
!unzip /content/BCICIV_2a_gdf.zip -d raw_data

#Installing Packages

In [ ]:
%%capture
!pip install mne

In [ ]:
%%capture
!pip install torch-summary

#Libraries Used

In [ ]:
import mne
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.autograd import Variable
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Structuring Data

In [ ]:
raw_data_folder = '/content/raw_data/'
cleaned_data_folder = '/content/cleaned_data/'
files = os.listdir(raw_data_folder)

# Filtering out files with suffix 'E.gdf'
filtered_files = [file for file in files if file.endswith('T.gdf')]

raw_list = []

# Iterating through filtered files
for file in filtered_files:
    file_path = os.path.join(raw_data_folder, file)

    # Reading raw data
    raw = mne.io.read_raw_gdf(file_path, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
    # Droping EOG channels
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])

    # High Pass Filtering 4Hz and above
    raw.filter(l_freq=4, h_freq=None)

    # Saving the modified raw data to a file with .fif suffix
    new_file_path = os.path.join(cleaned_data_folder, file[:-4] + '.fif')
    raw.save(new_file_path, overwrite=True)
    # Appending data to the list
    raw_list.append(raw)

final_raw = mne.concatenate_raws(raw_list)
new_file_path = os.path.join(cleaned_data_folder, 'All_Subjects.fif')
final_raw.save(new_file_path, overwrite=True)


**List of the events**  
'1023': 1 Rejected trial  
'1072': 2 Eye movements  
'276': 3 Idling EEG (eyes open)  
'277': 4 Idling EEG (eyes closed)  
'32766': 5 Start of a new run  
'768': 6 Start of a trial  
'769': 7 Cue onset **Left** (class 1) : 0  
'770': 8 Cue onset **Right** (class 2) : 1  
'771': 9 Cue onset **Foot** (class 3) : 2  
'772': 10 Cue onset **Tongue** (class 4): 3

In [ ]:
events = mne.events_from_annotations(final_raw)
events[1]


In [ ]:
epochs = mne.Epochs(final_raw, events[0], event_id=[7, 8, 9, 10], tmin=3.75, tmax=5.75, reject=None, baseline=None, preload=True)
data = epochs.get_data(copy=True)

In [ ]:
print("Dataset's shape:",data.shape)

In [ ]:
# Choosing Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loss Function
criterion = nn.CrossEntropyLoss()

# Normalizing Labels to [0, 1, 2, 3]
labels = epochs.events[:,-1]
min_label = np.min(labels)
y = labels - min_label

X = epochs.get_data(copy=True)

# Spliting  Data: 80% for Train and 20% for Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Converting to PyTorch Tensors
X_train = X_train.reshape((X_train.shape[0], -1, X_train.shape[-1]))
X_test = X_test.reshape((X_test.shape[0], -1, X_test.shape[-1]))
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
X_train = X_train.unsqueeze(1).to(device)
X_test = X_test.unsqueeze(1).to(device)
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)

# Printing the sizes
print("Size of X_train:", X_train.size())
print("Size of X_test:", X_test.size())
print("Size of y_train:", y_train.size())
print("Size of y_test:", y_test.size())


#EEGNet Model

In [ ]:
class EEGNet(nn.Module):
    def __init__(self,  chans=22, classes=4, kernLength=32, time_points=501,
                 f1=8, f2=16, d=2, dropoutRate=0.5, max_norm1=1, max_norm2=0.25):
        super(EEGNet, self).__init__()
        # Calculating FC input features
        linear_input_size = (((time_points)//8)//16)*f2

        # Temporal Filters
        self.block1 = nn.Sequential(
            nn.Conv2d(1, f1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(f1),
        )
        # Spatial Filters
        self.block2 = nn.Sequential(
            nn.Conv2d(f1, d * f1, (chans, 1), groups=f1, bias=False), # Depthwise Conv
            nn.BatchNorm2d(d * f1),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(dropoutRate)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(d * f1, f2, (1, 16), padding='same', bias=False), # Separable Conv
            nn.Conv2d(f2, f2, kernel_size=1), # Pointwise Conv
            nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d((1, 16)),
            nn.Dropout(dropoutRate)
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(linear_input_size, classes)

        # Apply max_norm constraint to the depthwise layer in block2
        self._apply_max_norm(self.block2[0], max_norm1)

        # Apply max_norm constraint to the linear layer
        self._apply_max_norm(self.fc, max_norm2)

    def _apply_max_norm(self, layer, max_norm):
        for name, param in layer.named_parameters():
            if 'weight' in name:
                param.data = torch.renorm(param.data, p=2, dim=0, maxnorm=max_norm)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


##Model Summery

In [ ]:
input_size = (1, 22, 501)
eegnet_model = EEGNet().to(device)
summary(eegnet_model, input_size)

##Training Loop

In [ ]:
eegnet_model = EEGNet().to(device)
learning_rate = 0.0001
optimizer = optim.Adam(eegnet_model.parameters(), lr=learning_rate)

num_epochs = 500
batch_size = 32
for epoch in range(num_epochs):
    eegnet_model.train()
    X_train, y_train = shuffle(X_train, y_train)
    running_loss = 0.0
    correct = 0
    total = 0
    for i in range(0, len(X_train), batch_size):
        inputs = X_train[i:i+batch_size].to(device)
        labels = y_train[i:i+batch_size].to(device)

        optimizer.zero_grad()
        outputs = eegnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(X_train)
    epoch_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {(epoch_accuracy*100):.2f}%")
average_loss = running_loss / len(X_train)
print("Average Loss:", average_loss)

# Saving model
torch.save(eegnet_model, 'eegnet_model.pth')


##Testing Model

In [ ]:
eegnet_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(X_test)):
        inputs = X_test[i:i+1].to(device)
        labels = y_test[i:i+1].to(device)
        outputs = eegnet_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = (correct / total)*100
print(f"Test Accuracy: {accuracy:.2f}%")


###Confusion Matrix

In [ ]:
eegnet_model.eval()
y_pred = []
y_true = []
classes = ['Left', 'Right', 'Foot', 'Tongue']

with torch.no_grad():
    for inputs, labels in zip(X_test, y_test):
        outputs = eegnet_model(inputs.unsqueeze(0))  # Forward pass
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted.item())
        y_true.append(labels.item())

cf_matrix = confusion_matrix(y_true, y_pred)
cf_matrix = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]

# Create DataFrame for visualization
df_cm = pd.DataFrame(cf_matrix, index=classes, columns=classes)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='.2f')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix_eegnet.png')
plt.show()
